In [ ]:
from __future__ import absolute_import, division, print_function

import os
import numpy as np
import PIL.Image as pil
import matplotlib.pyplot as plt
import glob
import torch
from torchvision import transforms
import cv2

In [ ]:
#Read the input images
input_path = "./*.png"
input_image_list = []
for im in sorted(glob.glob(input_path)):
    input_image = pil.open(im).convert('RGB')
    input_image_list.append(input_image)

In [ ]:
# Read the pielwise ground truth images
depth_gt_path = "./pixelwise_ground_truth/*.png"
ground_truth_magma = []
cntr = 0
for i in sorted(glob.glob(depth_gt_path)):
    im_gray = cv2.imread(i,cv2.IMREAD_GRAYSCALE)
    ground_truth_grey.append(im_gray)
    norm_img = np.zeros((256,256))
    final_img = cv2.normalize(im_gray, norm_img,0,255,cv2.NORM_MINMAX)
    im_color = cv2.applyColorMap(final_img,cv2.COLORMAP_MAGMA)
    ground_truth_magma.append(im_color)
    cntr+=1
    print(cntr)

In [ ]:
#Check if the input and ground truth image counts are consistent or not.
len(input_image_list),len(ground_truth_magma)

In [ ]:
#Check the grey scale and magma ground truths channel and pixel counts for first image
ground_truth_grey[0].shape, ground_truth_magma[0].shape

In [ ]:
#Give the path for the result of test_disp.py for input images. 
endosfm_pred_path = "./predictions.npy" 
endosfm_preds = np.load(endosfm_pred_path,allow_pickle=True)

In [ ]:
#Visualize the results as input ground truth and EndosfMLearner predictions
for i in range(len(endosfm_preds)):
    plt.figure(figsize=(12, 12))
    
    plt.subplot(131)
    plt.imshow(input_image_list[i])
    plt.title("Input", fontsize=9)
    plt.axis('off')

    plt.subplot(132)
    plt.imshow(ground_truth_grey[i], cmap='magma')
    plt.title("Ground Truth", fontsize=9)
    plt.axis('off')
    
    plt.subplot(133)
    plt.imshow(endosfm_preds[i], cmap='magma')
    plt.title("Predictions", fontsize=9)
    plt.axis('off')

In [ ]:
import numpy as np; np.random.seed(0)
import seaborn as sns; sns.set()
ax = sns.heatmap(ground_truth_grey[30]-endosfm_preds[30],annot=False,xticklabels=False,yticklabels=False,cbar=False)
ax.tick_params(left=False, bottom=False)

In [ ]:
# Pick an image with specific index to check min and max depth for image. As an example, we picked i=30.
print(np.amax(endosfm_preds[30])),print(np.amax(ground_truth_grey[30]))

In [ ]:
(endosfm_preds[30]-ground_truth_grey[30]).shape

In [ ]:
#ground truth depth vs. predicted depth calculations in terms of PSNR and SSIM metrics
import statistics 
import math
import cv2
from skimage.metrics import structural_similarity


psnrs = []
ssims = []

def psnr(img1, img2):
    mse = np.mean( (img1 - img2) ** 2 )
    if mse == 0:
        return 100
    PIXEL_MAX = 255.0
    return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))

def ssim(im1,im2):
    #grayA = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
    #grayB = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)
    (score, diff) = structural_similarity(im1, im2, full=True)
    return score


for i in range(len(ground_truth_magma)):
    gt_image = ground_truth_grey[i]
    pred_image = sfm_ours_preds[i].cpu().detach().numpy()
    p = psnr(gt_image,pred_image)
    s = ssim(gt_image,pred_image)
    psnrs.append(p)
    ssims.append(s)

psnr_mean_endo = statistics.mean(psnrs)
ssim_mean_endo = statistics.mean(ssims)
psnr_stdev_endo = statistics.stdev(psnrs)
ssim_stdev_endo = statistics.stdev(ssims)


In [ ]:
psnr_mean_endo, ssim_mean_endo, psnr_stdev_endo, ssim_stdev_endo

In [ ]:
# dump output image for psnrs as numpy array
np.save('./colon_depth_psnrs.npy',np.array(psnrs), allow_pickle=True)

# dump output image for ssims as numpy array
np.save('./colon_depth_ssim.npy',np.array(ssims), allow_pickle=True)

In [ ]:
import math
from skimage import img_as_float
from sklearn.metrics import mean_squared_error as mse
def rmsdiff(im1, im2):
    """Calculates the root mean square error (RSME) between two images"""
    return math.sqrt(mse(img_as_float(im1), img_as_float(im2)))

In [ ]:
def norm(image):
    normalized = (image - np.amin(image))/(np.amax(image)-np.amin(image))
    return normalized

In [ ]:
endosfm_rmse = []
for t in range(len(ground_truth_grey)):
    endosfm_rmse.append(rmsdiff(norm(ground_truth_grey[t]),norm(endosfm_preds[t])))               

In [ ]:
statistics.stdev(endosfm_rmse)
statistics.mean(endosfm_rmse)

In [ ]:
RM = PrettyTable()
RM.field_names = ["Algorithm","RMSE_mean","RMSE_std"]
RM.add_row(["EndoSfM","{:.4f}".format(statistics.mean(endosfm_rmse)),"{:.4f}".format(statistics.stdev(endosfm_rmse))])
print(RM)

In [ ]:
z = PrettyTable()
z.field_names = ["Algorithm", "TrainData", "TestData", "PSNR Mean", "SSIM Mean", "PSNR Stdev", "SSIM Stdev"]
for i in range(len(psnr_mean)):
    z.add_row(["EndoSfM", "Trained", "Stom-Unity", "{:.4f}".format(psnr_mean[i]),"{:.4f}".format(ssim_mean[i]), "{:.4f}".format(psnr_stdev[i]), "{:.4f}".format(ssim_stdev[i])])


In [ ]:
for i in range(len(ground_truth_grey)):
    gt_image = ground_truth_grey[i]
    pred_image = endosfm_preds[i]
    p = psnr(gt_image,pred_image)
    s = ssim(gt_image,pred_image)
    psnrs.append(p)
    ssims.append(s)

psnr_mean.append(statistics.mean(psnrs))
ssim_mean.append(statistics.mean(ssims))
psnr_stdev.append(statistics.stdev(psnrs))
ssim_stdev.append(statistics.stdev(ssims))